## Differential Expression Analysis
This script aims to conduct differential expression analysis using Wilcoxon Rank Sum and perhaps DESeq2. The experimental groups are as follows:
 - Noninfected - PTEN WT vs PTEN KO (How does PTEN loss change overall gene expression?)
 - Infected - PTEN WT vs. PTEN KO (How does the knockout respond differently to the WT when infected?)
 - Knockouts - Non-infected vs. Infected (How does the knockout respond with and without infection?)


In [7]:
#Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, cluster
import os

#umap
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#rna_seq analysis packages
import pickle as pkl
import pydeseq2 as deseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

### Data configuration

In [2]:
#Path configuration
wd = '/Users/nguyenpu/Documents/GitHub/'
os.chdir(wd)

datadir = "serezani_lab/bulk_rna_seq/pten_ko/data/"
outputdir = "serezani_lab/bulk_rna_seq/pten_ko/outputdir"

if os.path.isdir(outputdir)==False:
    os.mkdir(outputdir)

In [3]:
#Load datasets
uninfected_df = pd.read_csv(os.path.join(datadir, 'CTLUninfected_vs_KOUninfected.genes.counts.csv'))
infected_df = pd.read_csv(os.path.join(datadir, 'CTLInfected_vs_KOInfected.genes.counts.csv'))

In [4]:
#Load metadata
uninfected_md = pd.read_csv(os.path.join(datadir,'uninfected_metadata.csv'), index_col = 0)
infected_md = pd.read_csv(os.path.join(datadir,'infected_metadata.csv'), index_col = 0)

In [5]:
#Name EnsemblID Column
uninfected_df = pd.DataFrame.rename(uninfected_df, columns = {'Unnamed: 0':'Ensembl_ID'})
infected_df = pd.DataFrame.rename(infected_df, columns = {'Unnamed: 0':'Ensembl_ID'})

In [71]:
#Add GeneID column mapped to EnsemblID
gene_info = pd.read_csv(os.path.join(datadir, 'gene_info'))
ensembl2Gene_ID = gene_info[['Ensembl_ID', 'Gene_ID', 'Gene_Type']]

## Only run the below lines once
ensembl2Gene_ID[['Ensembl_ID', 'delete']] = gene_info['Ensembl_ID'].str.split('.', n=1, expand=True).copy()
ensembl2Gene_ID = ensembl2Gene_ID.drop('delete', axis = 1)


/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/19316353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ensembl2Gene_ID[['Ensembl_ID', 'delete']] = gene_info['Ensembl_ID'].str.split('.', n=1, expand=True).copy()
/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/19316353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ensembl2Gene_ID[['Ensembl_ID', 'delete']] = gene_info['Ensembl_ID'].str.split('.', n=1, expand=True).copy()


In [73]:
ensembl2Gene_ID

,Ensembl_ID,Gene_ID,Gene_Type
0,ENSMUSG00000094569,Trbd2,TR_D_gene
1,ENSMUSG00000095668,Trbd1,TR_D_gene
2,ENSMUSG00000096749,Trdd1,TR_D_gene
3,ENSMUSG00000096176,Trdd2,TR_D_gene
4,ENSMUSG00000094028,Ighd4-1,IG_D_gene
...,...,...,...
48951,ENSMUSG00000084750,Gm22453,misc_RNA
48952,ENSMUSG00000084711,Gm24629,misc_RNA
48953,ENSMUSG00000088014,Gm23743,misc_RNA
48954,ENSMUSG00000099219,Gm27680,misc_RNA


In [74]:
ensembl2Gene_ID.to_csv(os.path.join(datadir, 'ensembl2Gene_ID.csv'), index = False)

In [ ]:
# #Merge gene name and ensembl ID
# uninfected_df = ensembl2Gene_ID.merge(uninfected_df, on = ['Ensembl_ID'])
# infected_df = ensembl2Gene_ID.merge(infected_df,on = ['Ensembl_ID'])

In [ ]:
# #Drop the ensembl_ID column
# uninfected_df.drop(['Ensembl_ID'], axis = 1, inplace = True)
# infected_df.drop(['Ensembl_ID'], axis = 1, inplace = True)

In [12]:
#Make the Ensembl_ID column the index
uninfected_df.set_index('Ensembl_ID', inplace = True)
infected_df.set_index('Ensembl_ID', inplace = True)

In [13]:
#Transpose for DESeq2 analysis purposes
uninfected = uninfected_df.transpose()
infected = infected_df.transpose()

## DeSeq2

In [61]:
#Make function to perform DEseq2 analysis, so I don't need to type things twice
def deseq_analysis(count_df, metadata, outputdir, file_name, ensembl2Gene_ID):
    '''This function will perform analysis using DESeq2 given a count dataframe and metadata''' 
    #QC
    genes_to_keep = count_df.columns[count_df.sum(axis=0) >= 10]
    count_df = count_df[genes_to_keep]
    metadata['condition'] = metadata['condition'].astype(str)
    if 'condition' not in metadata.columns:
        raise KeyError("The 'condition' column is missing from metadata!")


    #Create a DeseqDataSet object from the count and metadata data that were just loaded
    inference = DefaultInference(n_cpus=6)
    dds = DeseqDataSet(
        counts=count_df,
        metadata=metadata,
        # ref_level = ['CTL', 'KO'],
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
    )

    ##Run DESeq2 Algorithm
    dds.deseq2()

    #Statistics
    stats_res = DeseqStats(dds, contrast=['condition','KO', 'CTL'], inference=inference) #contrast sets it so that it's WT vs. bft
    stats_res.summary()

    stats_res.lfc_shrink(coeff=None)
    deseq2_results = stats_res.results_df
    deseq2_results_no_NaN = deseq2_results.dropna(axis=0) #drop NA, there seems to be about 5000 genes that were dropped

    # Calculate the -log10(padj) and add to the data frame
    deseq2_results_no_NaN['-log10(padj)']=-np.log10(deseq2_results_no_NaN['padj'])
    deseq2_results_no_NaN['-log10(p-value)']=-np.log10(deseq2_results_no_NaN['pvalue'])

    #identify the genes in the dataframe that are upregulated, downregulated, or not at all
    deseq2_results_no_NaN['DEGenes']='NO' #make a new column for the DEgenes

    deseq2_results_no_NaN['DEGenes'].loc[(deseq2_results_no_NaN['log2FoldChange']>1) & (deseq2_results_no_NaN['padj']<0.05)] = 'UP'
    deseq2_results_no_NaN['DEGenes'].loc[(deseq2_results_no_NaN['log2FoldChange']<-1) & (deseq2_results_no_NaN['padj']<0.05)] = 'DOWN'

    #make and export a list of Ensembl_IDs that are upregulated and downregulated; doesn't contain additional gene annotation
    upreg = deseq2_results_no_NaN.loc[(deseq2_results_no_NaN['DEGenes']=='UP')]
    downreg = deseq2_results_no_NaN.loc[(deseq2_results_no_NaN['DEGenes']=='DOWN')]

    deseq2_results_no_NaN.to_csv(os.path.join(outputdir,file_name))
    return deseq2_results_no_NaN


In [83]:
#Run DESeq2 in terms of KO vs. CTL

uninfected_deseq2 = deseq_analysis(uninfected, uninfected_md, outputdir, 'DESeq2_CTLUninfected_vs_KOUninfected.csv', ensembl2Gene_ID)
infected_deseq2 = deseq_analysis(infected, infected_md, outputdir, 'DESeq2_CTLInfected_vs_KOInfected.csv', ensembl2Gene_ID)

Fitting size factors...
... done in 0.00 seconds.


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target

Log2 fold change & Wald test p-value: condition KO vs CTL
                       baseMean  log2FoldChange     lfcSE      stat  \
Ensembl_ID                                                            
ENSMUSG00000000001  9027.509719       -0.337508  0.096515 -3.496954   
ENSMUSG00000000028  1346.083231       -0.197182  0.118203 -1.668162   
ENSMUSG00000000056   923.275989        0.154391  0.132648  1.163911   
ENSMUSG00000000058  3078.829487        0.062695  0.122168  0.513186   
ENSMUSG00000000078  8236.341593       -0.577460  0.101141 -5.709427   
...                         ...             ...       ...       ...   
ENSMUSG00000118591    10.568534        2.904650  0.804464  3.610663   
ENSMUSG00000118607   202.616075        0.894560  0.189092  4.730818   
ENSMUSG00000118618     1.542217        3.970720  2.552247  1.555774   
ENSMUSG00000118619     4.196095       -0.031968  1.250840 -0.025557   
ENSMUSG00000118623     2.101578       -3.525727  2.016014 -1.748860   

                  

... done in 1.93 seconds.

/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/2962324808.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deseq2_results_no_NaN['-log10(padj)']=-np.log10(deseq2_results_no_NaN['padj'])
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/2962324808.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

Shrunk log2 fold change & Wald test p-value: condition KO vs CTL
                       baseMean  log2FoldChange     lfcSE      stat  \
Ensembl_ID                                                            
ENSMUSG00000000001  9027.509719       -0.337244  0.096110 -3.496954   
ENSMUSG00000000028  1346.083231       -0.190714  0.117412 -1.668162   
ENSMUSG00000000056   923.275989        0.145305  0.131513  1.163911   
ENSMUSG00000000058  3078.829487        0.057964  0.121195  0.513186   
ENSMUSG00000000078  8236.341593       -0.573360  0.100945 -5.709427   
...                         ...             ...       ...       ...   
ENSMUSG00000118591    10.568534        2.460562  0.799310  3.610663   
ENSMUSG00000118607   202.616075        0.857482  0.188980  4.730818   
ENSMUSG00000118618     1.542217        0.413443  1.739687  1.555774   
ENSMUSG00000118619     4.196095       -0.006363  0.758379 -0.025557   
ENSMUSG00000118623     2.101578       -0.457568  0.939469 -1.748860   

           

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.86 seconds.

Fitting dispersion trend curve...
... done in 0.20 seconds.

Fitting MAP dispersions...
... done in 2.05 seconds.

Fitting LFCs...
... done in 1.10 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 0.52 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: condition KO vs CTL
                       baseMean  log2FoldChange     lfcSE      stat  \
Ensembl_ID                                                            
ENSMUSG00000000001  8271.732962       -0.179003  0.066674 -2.684733   
ENSMUSG00000000028   982.445119       -0.271353  0.107660 -2.520472   
ENSMUSG00000000037     1.987273        1.590849  2.002825  0.794302   
ENSMUSG00000000056  1007.383977        0.685686  0.099740  6.874736   
ENSMUSG00000000058  2663.673954        0.390209  0.084888  4.596759   
...                         ...             ...       ...       ...   
ENSMUSG00000118590    72.775124        1.388921  0.282500  4.916535   
ENSMUSG00000118591    14.299576        3.213915  0.748050  4.296390   
ENSMUSG00000118607   301.326762        1.455751  0.185192  7.860785   
ENSMUSG00000118619     4.232791        0.352490  1.033357  0.341111   
ENSMUSG00000118640     2.485392        0.223911  1.624520  0.137832   

                  

... done in 2.20 seconds.

/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/2962324808.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deseq2_results_no_NaN['-log10(padj)']=-np.log10(deseq2_results_no_NaN['padj'])
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/k6/nqwv8_kx0zz_f6624qm3trb00000gp/T/ipykernel_53617/2962324808.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [88]:
ensembl2Gene_ID = pd.read_csv(os.path.join(datadir, 'ensembl2Gene_ID.csv'))

#Merge gene name and ensembl ID
uninfected_deseq2_final = ensembl2Gene_ID.merge(uninfected_deseq2, on = 'Ensembl_ID')
uninfected_deseq2_final.to_csv(os.path.join(outputdir, 'DESeq2_CTLUninfected_vs_KOUninfected.csv'), index = False)

infected_deseq2_final = ensembl2Gene_ID.merge(infected_deseq2, on = 'Ensembl_ID')
infected_deseq2_final.to_csv(os.path.join(outputdir, 'DESeq2_CTLInfected_vs_KOInfected.csv'), index = False)

In [93]:
#Make lists for upregulated and downregulated genes only
upreg_uninfected = uninfected_deseq2_final.loc[(uninfected_deseq2_final['DEGenes']=='UP')]
downreg_uninfected = uninfected_deseq2_final.loc[(uninfected_deseq2_final['DEGenes']=='DOWN')]

upreg_infected = infected_deseq2_final.loc[(infected_deseq2_final['DEGenes']=='UP')]
downreg_infected = infected_deseq2_final.loc[(infected_deseq2_final['DEGenes']=='DOWN')]

#Sort the dataframes by abs(log2FoldChange), greatest to least
upreg_uninfected = upreg_uninfected.sort_values(by = 'log2FoldChange', ascending = False)
downreg_uninfected = downreg_uninfected.sort_values(by = 'log2FoldChange', ascending = True)

upreg_infected = upreg_infected.sort_values(by = 'log2FoldChange', ascending = False)
downreg_infected = downreg_infected.sort_values(by = 'log2FoldChange', ascending = True)

#Save the upregulated and downregulated genes
upreg_uninfected.to_csv(os.path.join(outputdir, 'upreg_uninfected.csv'), index = False)
downreg_uninfected.to_csv(os.path.join(outputdir, 'downreg_uninfected.csv'), index = False)

upreg_infected.to_csv(os.path.join(outputdir, 'upreg_infected.csv'), index = False)
downreg_infected.to_csv(os.path.join(outputdir, 'downreg_infected.csv'), index = False)